In [1]:
# end_year 컬럼 추가

# 전자결재 목록 크롤러 (간편 버전)
# 상세 진입 없이 목록(그리드)만 수집
# 중단점 저장 + 재시작 지원 + JSON + DB 저장

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pathlib import Path
from sqlalchemy import create_engine, text
from sqlalchemy.types import BIGINT, VARCHAR
import time
import json
import traceback
import re

# ═══════════════════════════════════════════════════════════
# 🔧 설정 (여기만 수정하세요!)
# ═══════════════════════════════════════════════════════════

BASE_URL = "http://office.anyfive.com/home/"

# 기간 설정
# 5년 이하 경고창떠서 두번 쪼개서 실행함
END_YEAR = 2015  #컨트롤러 - 폴더 기간에 맞게 수정 (2025, 2020, 2015, 2010)
START_DATE = "2011-01-01"  #컨트롤러 - 시작일
END_DATE = "2015-12-31"  #컨트롤러 - 종료일



# DB 설정
DB_CONFIG = {
    "user": "root",
    "password": "1234",
    "host": "127.0.0.1",
    "port": 3306,
    "database": "any_approval" #컨트롤러
}

# 저장 파일명
JSON_FILE = "one_formCrawling_result.json" #컨트롤러
CHECKPOINT_FILE = "one_formCrawling_checkpoint.json" #컨트롤러

# ═══════════════════════════════════════════════════════════
# 📦 유틸리티 함수
# ═══════════════════════════════════════════════════════════

def norm_space(s):
    """공백 정규화"""
    if not s: return ""
    s = s.replace("\xa0", " ")
    s = re.sub(r'[\t\r\n]+', ' ', s)
    s = re.sub(r'\s+', ' ', s)
    return s.strip()

def switch_into_frame_having(driver, selector_css, max_retries=3):
    """selector가 있는 프레임으로 전환"""
    for attempt in range(max_retries):
        driver.switch_to.default_content()
        
        # 로딩 마스크 대기
        try:
            WebDriverWait(driver, 5).until(
                EC.invisibility_of_element_located((By.CSS_SELECTOR, ".k-loading-mask"))
            )
        except:
            pass
        
        time.sleep(1)
        
        # 최상위 확인
        if driver.find_elements(By.CSS_SELECTOR, selector_css):
            print(f"  ✓ 그리드 발견 (최상위)")
            return True
        
        # 프레임 내부 탐색
        frames = driver.find_elements(By.CSS_SELECTOR, "iframe, frame")
        for idx, fr in enumerate(frames):
            try:
                driver.switch_to.default_content()
                driver.switch_to.frame(fr)
                if driver.find_elements(By.CSS_SELECTOR, selector_css):
                    print(f"  ✓ 그리드 발견 (프레임 {idx})")
                    return True
            except:
                continue
        
        if attempt < max_retries - 1:
            print(f"  ⚠️ 재시도 {attempt + 1}/{max_retries}")
            time.sleep(2)
    
    return False

def wait_grid_stable(driver, timeout=10):
    """그리드 안정화 대기"""
    try:
        WebDriverWait(driver, timeout).until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR, ".k-loading-mask"))
        )
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "table[role='grid'] tbody tr"))
        )
        time.sleep(1)
        return True
    except:
        return False

# ═══════════════════════════════════════════════════════════
# 💾 체크포인트 관리
# ═══════════════════════════════════════════════════════════

def load_checkpoint():
    """체크포인트 로드"""
    if Path(CHECKPOINT_FILE).exists():
        with open(CHECKPOINT_FILE, 'r', encoding='utf-8') as f:
            return json.load(f)
    return {"last_page": 0, "total_collected": 0, "last_doc_id": None}

def save_checkpoint(page, total, last_doc_id):
    """체크포인트 저장"""
    checkpoint = {
        "last_page": page,
        "total_collected": total,
        "last_doc_id": last_doc_id,
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
    }
    with open(CHECKPOINT_FILE, 'w', encoding='utf-8') as f:
        json.dump(checkpoint, f, ensure_ascii=False, indent=2)
    print(f"  💾 체크포인트 저장: 페이지 {page}, 총 {total}건")

# ═══════════════════════════════════════════════════════════
# 💾 데이터 저장
# ═══════════════════════════════════════════════════════════

def load_existing_data():
    """기존 JSON 데이터 로드"""
    if Path(JSON_FILE).exists():
        with open(JSON_FILE, 'r', encoding='utf-8') as f:
            return json.load(f)
    return {}

def save_to_json(data_dict):
    """JSON 파일에 저장 (병합 방식)"""
    existing = load_existing_data()
    existing.update(data_dict)  # 병합!
    
    with open(JSON_FILE, 'w', encoding='utf-8') as f:
        json.dump(existing, f, ensure_ascii=False, indent=2)
    
    print(f"  📄 JSON 저장: {len(existing)}건 (새로 추가 {len(data_dict)}건)")

def save_to_db(data_list, engine, end_year):
    """DB에 저장 (UPSERT 방식)"""
    if not data_list:
        return
    
    # 테이블 생성 (없으면) 
    create_table_sql = """
    CREATE TABLE IF NOT EXISTS form (
        문서ID BIGINT PRIMARY KEY,
        양식명 VARCHAR(255),
        문서번호 VARCHAR(255),
        제목 VARCHAR(500),
        기안자 VARCHAR(255),
        기안일 VARCHAR(50),
        end_year INT,
        INDEX idx_end_year (end_year)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
    """
    
    with engine.connect() as conn:
        conn.execute(text(create_table_sql))
        conn.commit()
    
    # UPSERT (ON DUPLICATE KEY UPDATE)
    upsert_sql = """
    INSERT INTO form 
        (문서ID, 양식명, 문서번호, 제목, 기안자, 기안일, end_year)
    VALUES 
        (:doc_id, :form_name, :doc_num, :title, :author, :draft_date, :end_year)
    ON DUPLICATE KEY UPDATE
        양식명 = VALUES(양식명),
        문서번호 = VALUES(문서번호),
        제목 = VALUES(제목),
        기안자 = VALUES(기안자),
        기안일 = VALUES(기안일),
        end_year = VALUES(end_year)
    """
    
    with engine.connect() as conn:
        for item in data_list:
            conn.execute(text(upsert_sql), {
                "doc_id": item["문서ID"],
                "form_name": item["양식명"],
                "doc_num": item["문서번호"],
                "title": item["제목"],
                "author": item["기안자"],
                "draft_date": item["기안일"],
                "end_year": end_year
            })
        conn.commit()
    
    print(f"  💾 DB 저장: {len(data_list)}건")

# ═══════════════════════════════════════════════════════════
# 🕷️ 그리드 파싱
# ═══════════════════════════════════════════════════════════

def parse_grid_rows(driver, end_year):
    """현재 페이지의 그리드 파싱"""
    table = driver.find_element(By.CSS_SELECTOR, "table[role='grid']")
    
    # 헤더 분석
    th_all = table.find_elements(By.CSS_SELECTOR, "thead th")
    field_index = {}
    for i, th in enumerate(th_all):
        field = (th.get_attribute("data-field") or "").strip()
        if field:
            field_index[field] = i
    
    # 행 파싱
    rows = table.find_elements(By.CSS_SELECTOR, "tbody tr[role='row']")
    results = []
    
    for tr in rows:
        tds = tr.find_elements(By.CSS_SELECTOR, "td")
        if not tds:
            continue
        
        # 체크박스에서 문서ID 추출
        cb = tr.find_elements(By.CSS_SELECTOR, "td input[name='aprDoc_grid_ckbx']")
        doc_id = cb[0].get_attribute("value").strip() if cb else ""
        
        def get_td_text(field, fallback_idx=None):
            """field 이름이나 인덱스로 td 텍스트 추출"""
            idx = field_index.get(field)
            if idx is not None and idx < len(tds):
                return norm_space(tds[idx].text or tds[idx].get_attribute("textContent") or "")
            if fallback_idx is not None and fallback_idx < len(tds):
                return norm_space(tds[fallback_idx].text or tds[fallback_idx].get_attribute("textContent") or "")
            return ""
        
        item = {
            "문서ID": doc_id,
            "양식명": get_td_text("form_nm", 1),
            "문서번호": get_td_text("doc_sn", 2),
            "제목": get_td_text("appr_doc_tit", 3),
            "기안자": get_td_text("kor_nm", 4),
            "기안일": get_td_text("drft_dtm", 5),
            "end_year": end_year
        }
        
        results.append(item)
    
    return results

# ═══════════════════════════════════════════════════════════
# 🔄 페이징
# ═══════════════════════════════════════════════════════════

def has_next_page(driver):
    """다음 페이지 존재 여부"""
    try:
        next_btns = driver.find_elements(By.CSS_SELECTOR, 
            "a.k-pager-nav[aria-label='다음 페이지로 이동']:not(.k-state-disabled)")
        return len(next_btns) > 0
    except:
        return False

def go_to_next_page(driver):
    """다음 페이지로 이동"""
    try:
        next_btn = driver.find_element(By.CSS_SELECTOR,
            "a.k-pager-nav[aria-label='다음 페이지로 이동']:not(.k-state-disabled)")
        
        driver.execute_script("arguments[0].click();", next_btn)
        time.sleep(2)
        
        # 로딩 대기
        WebDriverWait(driver, 15).until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR, ".k-loading-mask"))
        )
        
        wait_grid_stable(driver)
        return True
    except Exception as e:
        print(f"  ❌ 페이지 이동 실패: {e}")
        return False

def go_to_specific_page(driver, target_page):
    """특정 페이지로 이동 (재시작용)"""
    if target_page <= 1:
        return True
    
    print(f"  🔄 {target_page}페이지로 이동 중...")
    
    for page in range(2, target_page + 1):
        if not has_next_page(driver):
            print(f"  ⚠️ {page}페이지에 도달하지 못함")
            return False
        
        if not go_to_next_page(driver):
            return False
        
        if page % 5 == 0:
            print(f"    진행 중... {page}/{target_page}")
    
    print(f"  ✓ {target_page}페이지 도착")
    return True

# ═══════════════════════════════════════════════════════════
# 🚀 메인 크롤러
# ═══════════════════════════════════════════════════════════

def main():
    print("=" * 70)
    print("🕷️  전자결재 목록 크롤러 시작")
    print(f"📅 기간: {START_DATE} ~ {END_DATE} (endYear={END_YEAR})")
    print("=" * 70)
    
    # 체크포인트 로드
    checkpoint = load_checkpoint()
    start_page = checkpoint["last_page"] + 1
    total_collected = checkpoint["total_collected"]
    
    if start_page > 1:
        print(f"\n🔄 재시작 모드")
        print(f"  마지막 완료 페이지: {checkpoint['last_page']}")
        print(f"  기존 수집량: {total_collected}건")
        print(f"  {start_page}페이지부터 이어서 시작합니다.\n")
    else:
        print(f"\n🆕 새로 시작\n")
    
    # DB 연결
    try:
        engine = create_engine(
            f"mysql+pymysql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@"
            f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
            f"?charset=utf8mb4"
        )
        print(f"✅ DB 연결 성공: {DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}\n")
    except Exception as e:
        print(f"❌ DB 연결 실패: {e}")
        return
    
    # 브라우저 시작
    opts = webdriver.ChromeOptions()
    opts.add_argument("--start-maximized")
    opts.add_argument("--disable-popup-blocking")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    
    driver = webdriver.Chrome(service=Service(), options=opts)
    
    try:
        # 로그인
        driver.get(BASE_URL)
        time.sleep(1)
        
        print("\n" + "=" * 70)
        print("📋 수동 작업이 필요합니다!")
        print("=" * 70)
        print("다음 단계를 완료한 후 Enter를 눌러주세요:\n")
        print("  1️⃣  로그인")
        print("  2️⃣  전자결재 메뉴 클릭")
        print("  3️⃣  결재 문서관리 클릭")
        print(f"  4️⃣  상태 = 완료 선택")
        print(f"  5️⃣  시작일 = {START_DATE}")
        print(f"  6️⃣  종료일 = {END_DATE}")
        print("  7️⃣  조회 버튼 클릭 → 그리드 나타남 확인")
        print("\n💡 그리드가 화면에 보이면 Enter를 누르세요!")
        print("=" * 70)
        input("\n✋ 준비되면 Enter... ")
        
        time.sleep(1)
        
        # 그리드로 전환
        if not switch_into_frame_having(driver, "table[role='grid']"):
            print("❌ 그리드를 찾을 수 없습니다")
            return
        
        wait_grid_stable(driver)
        
        # 재시작 시 특정 페이지로 이동
        if start_page > 1:
            if not go_to_specific_page(driver, start_page):
                print("❌ 시작 페이지로 이동 실패")
                return
        
        # 페이지별 수집
        current_page = start_page
        
        while True:
            print(f"\n{'=' * 70}")
            print(f"📄 페이지 {current_page} 수집 중...")
            print(f"{'=' * 70}")
            
            try:
                # 그리드 재확인
                if not switch_into_frame_having(driver, "table[role='grid']"):
                    print("⚠️ 그리드 재탐색 실패")
                    break
                
                wait_grid_stable(driver)
                
                # 파싱
                page_data = parse_grid_rows(driver, END_YEAR)
                
                if not page_data:
                    print("⚠️ 데이터가 없습니다")
                    break
                
                print(f"  ✓ {len(page_data)}건 파싱 완료")
                
                # JSON 저장 (병합)
                json_dict = {item["문서ID"]: {
                    "양식명": item["양식명"],
                    "문서번호": item["문서번호"],
                    "제목": item["제목"],
                    "기안자": item["기안자"],
                    "기안일": item["기안일"],
                    "end_year": item["end_year"]
                } for item in page_data}
                
                save_to_json(json_dict)
                
                # DB 저장 (UPSERT)
                save_to_db(page_data, engine, END_YEAR)
                
                # 체크포인트 저장
                total_collected += len(page_data)
                last_doc_id = page_data[-1]["문서ID"] if page_data else None
                save_checkpoint(current_page, total_collected, last_doc_id)
                
                print(f"  ✅ 페이지 {current_page} 완료 (누적: {total_collected}건)")
                
                # 다음 페이지
                if has_next_page(driver):
                    if go_to_next_page(driver):
                        current_page += 1
                        time.sleep(1)
                    else:
                        break
                else:
                    print(f"\n🎉 마지막 페이지 도달!")
                    break
                
            except Exception as e:
                print(f"❌ 페이지 {current_page} 처리 실패: {e}")
                traceback.print_exc()
                break
        
        print(f"\n{'=' * 70}")
        print(f"✅ 수집 완료!")
        print(f"{'=' * 70}")
        print(f"📊 총 수집: {total_collected}건")
        print(f"📄 JSON: {JSON_FILE}")
        print(f"💾 DB: form 테이블 (endYear={END_YEAR})")
        print(f"📍 체크포인트: {CHECKPOINT_FILE}")
        print(f"{'=' * 70}\n")
        
    except Exception as e:
        print(f"\n❌ 오류 발생: {e}")
        traceback.print_exc()
    
    finally:
        input("\n✋ 브라우저를 닫으려면 Enter를 누르세요...")
        driver.quit()

if __name__ == "__main__":
    main()

🕷️  전자결재 목록 크롤러 시작
📅 기간: 2011-01-01 ~ 2015-12-31 (endYear=2015)

🆕 새로 시작

✅ DB 연결 성공: 127.0.0.1:3306/any_approval


📋 수동 작업이 필요합니다!
다음 단계를 완료한 후 Enter를 눌러주세요:

  1️⃣  로그인
  2️⃣  전자결재 메뉴 클릭
  3️⃣  결재 문서관리 클릭
  4️⃣  상태 = 완료 선택
  5️⃣  시작일 = 2011-01-01
  6️⃣  종료일 = 2015-12-31
  7️⃣  조회 버튼 클릭 → 그리드 나타남 확인

💡 그리드가 화면에 보이면 Enter를 누르세요!



✋ 준비되면 Enter...  


  ✓ 그리드 발견 (프레임 1)

📄 페이지 1 수집 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 파싱 완료
  📄 JSON 저장: 50건 (새로 추가 50건)
  💾 DB 저장: 50건
  💾 체크포인트 저장: 페이지 1, 총 50건
  ✅ 페이지 1 완료 (누적: 50건)

📄 페이지 2 수집 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 파싱 완료
  📄 JSON 저장: 100건 (새로 추가 50건)
  💾 DB 저장: 50건
  💾 체크포인트 저장: 페이지 2, 총 100건
  ✅ 페이지 2 완료 (누적: 100건)

📄 페이지 3 수집 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 파싱 완료
  📄 JSON 저장: 150건 (새로 추가 50건)
  💾 DB 저장: 50건
  💾 체크포인트 저장: 페이지 3, 총 150건
  ✅ 페이지 3 완료 (누적: 150건)

📄 페이지 4 수집 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 파싱 완료
  📄 JSON 저장: 200건 (새로 추가 50건)
  💾 DB 저장: 50건
  💾 체크포인트 저장: 페이지 4, 총 200건
  ✅ 페이지 4 완료 (누적: 200건)

📄 페이지 5 수집 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 파싱 완료
  📄 JSON 저장: 250건 (새로 추가 50건)
  💾 DB 저장: 50건
  💾 체크포인트 저장: 페이지 5, 총 250건
  ✅ 페이지 5 완료 (누적: 250건)

📄 페이지 6 수집 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 파싱 완료
  📄 JSON 저장: 300건 (새로 추가 50건)
  💾 DB 저장: 50건
  💾 체크포인트 저장: 페이지 6, 총 300건
  ✅ 페이지 6 완료 (누적: 300건)

📄 페이지 7 수집 중...
  ✓ 그리드 발견 (프레임 1)
  ✓ 50건 파싱 완료
  📄 JSON 저장: 350건 (새로 추가 50건)
  💾 DB 저장: 50건
  💾 체크포


✋ 브라우저를 닫으려면 Enter를 누르세요... 
